In [16]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader("data/")
    return document_loader.load()

In [17]:
documents = load_documents()

In [32]:
print(documents[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents : list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100,
        length_function = len,
        is_separator_regex = False,
    )
    
    return text_splitter.split_documents(documents)


In [40]:
chunks = split_documents(documents)
print(chunks[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}
